## Iterator (ang. Iterator, Cursor)

**Typ**: behawioralny \
**Zakres**: obiektowy

<div style="border: solid 1px;padding: 20px;text-align: center">
    Wzorzec <b>iterator</b> zapewnia sposób na sekwencyjny dostęp do elementów agregatu bez ujawniania jego wewnętrznej reprezentacji.
</div>

### Problem - dostęp do kolekcji bez ujawniania struktury

Masz kolekcję książek. Klient chce przejść przez wszystkie książki.

**Jak to zrobić bez:**
- Ujawniania struktury wewnętrznej (lista? drzewo? hash?)
- Zaśmiecania kolekcji logiką iteracji
- Ograniczania do jednego sposobu iteracji

### Naiwne podejście - bezpośredni dostęp do struktury

In [ ]:
class BookCollection:
    def __init__(self):
        self.books = []  # ❌ Struktura wewnętrzna ujawniona!
    
    def add(self, book):
        self.books.append(book)

In [ ]:
collection = BookCollection()
collection.add("Python 101")
collection.add("Design Patterns")
collection.add("Clean Code")

# Klient musi znać strukturę wewnętrzną!
for book in collection.books:  # ❌ Bezpośredni dostęp do `books`
    print(book)

**Problemy:**
- ❌ Klient **zna strukturę** (`books` to lista)
- ❌ Co jeśli zmienisz `list` na `dict` albo drzewo? **Klient się zepsuje**!
- ❌ Nie możesz mieć **wielu iteracji** jednocześnie
- ❌ **Brak hermetyzacji** - klient może modyfikować `books` bezpośrednio

### Rozwiązanie - wzorzec Iterator (klasyczny)

**Idea:** Osobny obiekt **Iterator** odpowiada za przechodzenie przez kolekcję.

### Krok 1: Interfejs Iterator

In [ ]:
from abc import ABC, abstractmethod

class Iterator(ABC):
    """Interfejs iteratora"""
    
    @abstractmethod
    def has_next(self) -> bool:
        """Czy są jeszcze elementy?"""
        pass
    
    @abstractmethod
    def next(self):
        """Zwróć następny element"""
        pass

### Krok 2: Konkretny Iterator

In [ ]:
class BookIterator(Iterator):
    """Iterator dla kolekcji książek"""
    
    def __init__(self, books):
        self._books = books  # Dostęp do kolekcji
        self._index = 0      # Aktualny indeks
    
    def has_next(self) -> bool:
        return self._index < len(self._books)
    
    def next(self):
        if not self.has_next():
            raise StopIteration("Koniec kolekcji")
        
        book = self._books[self._index]
        self._index += 1
        return book

### Krok 3: Agregat (kolekcja) zwracający Iterator

In [ ]:
class BookCollection:
    """Kolekcja książek - ukrywa strukturę wewnętrzną"""
    
    def __init__(self):
        self._books = []  # ✅ Prywatne - klient nie ma dostępu!
    
    def add(self, book):
        self._books.append(book)
    
    def create_iterator(self) -> Iterator:
        """Tworzy iterator dla tej kolekcji"""
        return BookIterator(self._books)

**Kluczowa zmiana:**
- Kolekcja **NIE ujawnia** `_books`
- Zamiast tego zwraca **Iterator**
- Klient używa `has_next()` i `next()` - **nie zna struktury**!

### Krok 4: Użycie - bez znajomości struktury

In [ ]:
collection = BookCollection()
collection.add("Python 101")
collection.add("Design Patterns")
collection.add("Clean Code")

# ✅ Klient NIE zna struktury wewnętrznej
iterator = collection.create_iterator()

while iterator.has_next():
    book = iterator.next()
    print(book)

**Zalety:**
- ✅ **Hermetyzacja** - struktura ukryta
- ✅ **Możesz zmienić** `list` na `dict` - klient nie zauważy
- ✅ **Wiele iteracji** jednocześnie (każdy iterator niezależny)
- ✅ **Oddzielenie odpowiedzialności** - kolekcja vs iteracja

## Wiele iteracji jednocześnie

In [ ]:
collection = BookCollection()
collection.add("Book A")
collection.add("Book B")
collection.add("Book C")

# Dwa niezależne iteratory!
iter1 = collection.create_iterator()
iter2 = collection.create_iterator()

print("Iter1:", iter1.next())  # Book A
print("Iter2:", iter2.next())  # Book A
print("Iter1:", iter1.next())  # Book B
print("Iter2:", iter2.next())  # Book B

# Każdy iterator ma swój własny indeks!

## Struktura wzorca (klasyczna)

**Elementy wzorca Iterator:**

1. **Iterator** - `Iterator`
   - Interfejs dla iteratorów
   - Metody: `has_next()`, `next()`

2. **ConcreteIterator** - `BookIterator`
   - Konkretny iterator
   - Przechowuje aktualną pozycję
   - Wie jak przejść przez kolekcję

3. **Aggregate** - interfejs kolekcji
   - Deklaruje `create_iterator()`

4. **ConcreteAggregate** - `BookCollection`
   - Konkretna kolekcja
   - Tworzy konkretny iterator

## Iterator w Pythonie - protokół iteratora

Python ma **wbudowany protokół iteratora**: `__iter__()` i `__next__()`

**Przeniesienie języka wzorca na język Pythonowy:**
- `create_iterator()` → `__iter__()`
- `has_next()` + `next()` → `__next__()` + `StopIteration`
- `while iterator.has_next()` → `for item in collection`

### Pythonowy Iterator - wersja 1 (osobna klasa iteratora)

In [ ]:
class BookIterator:
    """Iterator - implementuje __next__()"""
    
    def __init__(self, books):
        self._books = books
        self._index = 0
    
    def __iter__(self):
        # Iterator zwraca sam siebie
        return self
    
    def __next__(self):
        if self._index >= len(self._books):
            raise StopIteration  # Koniec iteracji
        
        book = self._books[self._index]
        self._index += 1
        return book


class BookCollection:
    """Kolekcja - implementuje __iter__()"""
    
    def __init__(self):
        self._books = []
    
    def add(self, book):
        self._books.append(book)
    
    def __iter__(self):
        # Zwraca nowy iterator
        return BookIterator(self._books)

In [ ]:
collection = BookCollection()
collection.add("Python 101")
collection.add("Design Patterns")
collection.add("Clean Code")

# ✅ Działa z pętlą for!
for book in collection:
    print(book)

**Jak to działa:**
```python
for book in collection:
    print(book)
```

Python robi:
1. `iterator = collection.__iter__()` → tworzy `BookIterator`
2. `book = iterator.__next__()` → pierwszy element
3. `book = iterator.__next__()` → drugi element
4. ...
5. `iterator.__next__()` → `StopIteration` → koniec pętli

### Pythonowy Iterator - wersja 2 (generator z `yield`)

In [ ]:
class BookCollection:
    """Używa generatora - najprostsze rozwiązanie!"""
    
    def __init__(self):
        self._books = []
    
    def add(self, book):
        self._books.append(book)
    
    def __iter__(self):
        # Generator - yield tworzy iterator automatycznie!
        for book in self._books:
            yield book

In [ ]:
collection = BookCollection()
collection.add("Python 101")
collection.add("Design Patterns")
collection.add("Clean Code")

# ✅ Działa identycznie!
for book in collection:
    print(book)

**Generator (`yield`):**
- Python **automatycznie tworzy** iterator
- **Nie musisz** pisać osobnej klasy `BookIterator`
- Prostsze, krótsze, idiomatyczne dla Pythona

## Przykład - różne sposoby iteracji

In [ ]:
class BookCollection:
    def __init__(self):
        self._books = []
    
    def add(self, book):
        self._books.append(book)
    
    def __iter__(self):
        """Domyślna iteracja - od początku do końca"""
        for book in self._books:
            yield book
    
    def reverse_iterator(self):
        """Iteracja od końca"""
        for book in reversed(self._books):
            yield book
    
    def every_second(self):
        """Co drugi element"""
        for i in range(0, len(self._books), 2):
            yield self._books[i]

In [ ]:
collection = BookCollection()
collection.add("Book 1")
collection.add("Book 2")
collection.add("Book 3")
collection.add("Book 4")

print("Normalna iteracja:")
for book in collection:
    print(book)

print("\nOd końca:")
for book in collection.reverse_iterator():
    print(book)

print("\nCo drugi:")
for book in collection.every_second():
    print(book)

## Porównanie: Klasyczny wzorzec vs Python

| Aspekt | Klasyczny Iterator | Iterator Pythonowy |
|--------|-------------------|--------------------|
| **Metody** | `has_next()`, `next()` | `__iter__()`, `__next__()` |
| **Koniec iteracji** | `has_next()` zwraca `False` | `__next__()` rzuca `StopIteration` |
| **Tworzenie** | `collection.create_iterator()` | `iter(collection)` lub `for` |
| **Pętla** | `while iterator.has_next()` | `for item in collection` |
| **Klasa iteratora** | Zawsze osobna klasa | Opcjonalna (można użyć `yield`) |
| **Idiom** | Uniwersalny (Java, C++) | Pythonowy (`yield`, generatory) |

### Klasyczny wzorzec (uniwersalny)

In [ ]:
# Tworzenie iteratora
iterator = collection.create_iterator()

# Pętla
while iterator.has_next():
    item = iterator.next()
    print(item)

### Python - wersja 1 (`__iter__` + osobna klasa)

In [ ]:
# Iterator jako osobna klasa
class BookIterator:
    def __init__(self, books):
        self._books = books
        self._index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index >= len(self._books):
            raise StopIteration
        book = self._books[self._index]
        self._index += 1
        return book

class BookCollection:
    def __iter__(self):
        return BookIterator(self._books)

# Użycie
for book in collection:
    print(book)

### Python - wersja 2 (`yield` - idiomatyczna)

In [ ]:
# Generator z yield - najprostsze!
class BookCollection:
    def __iter__(self):
        for book in self._books:
            yield book  # Python tworzy iterator automatycznie

# Użycie - identyczne
for book in collection:
    print(book)

## Kiedy używać wzorca Iterator?

Wzorzec Iterator stosuj gdy:

1. **Chcesz ukryć strukturę wewnętrzną kolekcji**
   - Klient nie powinien znać czy to lista, drzewo czy hash

2. **Potrzebujesz wielu sposobów iteracji**
   - Od początku, od końca, co drugi element

3. **Chcesz jednolity interfejs dla różnych kolekcji**
   - Lista, drzewo, graf - ten sam sposób iteracji

4. **Potrzebujesz wielu niezależnych iteracji jednocześnie**
   - Każdy iterator ma swoją pozycję

**W Pythonie:**
- Używaj `__iter__()` + `yield` (najprostsze)
- Osobna klasa iteratora tylko gdy:
  - Iterator ma skomplikowany stan
  - Iterator wykonuje dodatkową logikę

**Przykłady praktyczne:**
- Kolekcje niestandardowe (drzewo, graf)
- Lazy loading (elementy ładowane w locie)
- Strumienie danych (pliki, sieć)
- Range, enumerate, zip w Pythonie

## Podsumowanie

Wzorzec Iterator:
- ✅ **Ukrywa strukturę** kolekcji
- ✅ **Jednolity interfejs** do przechodzenia kolekcji
- ✅ **Wiele iteracji** jednocześnie (niezależne iteratory)
- ✅ **Różne sposoby** iteracji (forward, reverse, filtrowanie)
- ✅ **Separacja odpowiedzialności** - kolekcja vs iteracja
- ⚠️ **Overkill** dla prostych list (w Pythonie wystarczy `for item in list`)

**Kluczowa idea:**
> Dostęp sekwencyjny do kolekcji **bez ujawniania** jej struktury

**Klasyczna struktura:**
- **Iterator** - interfejs (`has_next()`, `next()`)
- **ConcreteIterator** - konkretny iterator (przechowuje indeks)
- **Aggregate** - kolekcja (`create_iterator()`)

**Pythonowa struktura:**
```python
class Collection:
    def __iter__(self):
        for item in self._items:
            yield item  # Generator!

# Użycie
for item in collection:
    print(item)
```

**Przeniesienie na język Pythona:**

| Wzorzec | Python |
|---------|--------|
| `create_iterator()` | `__iter__()` |
| `has_next()` + `next()` | `__next__()` + `StopIteration` |
| `while iterator.has_next()` | `for item in collection` |
| Osobna klasa iteratora | Generator z `yield` |

**Istota wzorca:**
- **Hermetyzacja:** Klient nie zna struktury (`_books` ukryte)
- **Niezależność:** Każdy iterator ma swój stan (indeks)
- **Polimorfizm:** Ten sam interfejs dla różnych kolekcji